 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd
import synonyms
import jieba
import jieba.posseg as pseg

Model loaded succeed
>> Synonyms on loading ...
>> Synonyms vocabulary size: 125792


In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
corpus_df = pd.read_csv('/Users/olap/Desktop/corpus.csv')
corpus_2_list = corpus_df[corpus_df['rate'] == 2]['token'].tolist()
corpus_3_list = corpus_df[corpus_df['rate'] == 3]['token'].tolist()

In [4]:
def title_token_rater(tokens):
    
    tokens_rate = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), tokens))
        
    token_3_cnt = tokens_rate.count(3)
    token_2_cnt = tokens_rate.count(2)
    
    return [token_2_cnt, token_3_cnt]

In [5]:
def title_syn_rater(tokens):
    
    syn_3_cnt = 0
    syn_2_cnt = 0
    syn_cnt_list = []
    
    for token in tokens:
        syn = synonyms.nearby(token)[0]
        syn_rates = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), syn))

        syn_cnt = [2 if syn_rates.count(3) < syn_rates.count(2) else(0 if syn_rates.count(3) == 0 and syn_rates.count(2) == 0 else 3)]
        syn_cnt_list += syn_cnt
        
    syn_3_cnt = syn_cnt_list.count(3)
    syn_2_cnt = syn_cnt_list.count(2)
     
    return [syn_2_cnt, syn_3_cnt]

In [6]:
def title_processor(title):
    
    tokens = list(set(list(jieba.cut_for_search(title))))
#     print(tokens)
    token_rate = title_token_rater(tokens)
    syn_rate = title_syn_rater(tokens)
    
    return token_rate + syn_rate

In [8]:
training_title_list = open('/Users/olap/Desktop/training_title_list.txt', 'r').read().split('\n')[:100]

In [18]:
test_title = training_title_list[2]
print (test_title)

跟老公结婚当天，老公强行的让我和他的几个朋友一起爱爱，我是第一次，下面疼的几乎晕过去，屁眼也被插了，_飞华健康问答


In [19]:
title_processor(test_title)

[2, 1, 2, 0]

In [24]:
training_title_list = open('/Users/olap/Desktop/training_title_list.txt', 'r').read().split('\n')[:100]
title_feature_list = list(map(title_processor, training_title_list))

ture_label = open('/Users/olap/Desktop/training_set.txt', 'r').read().split('\n')
title_rate = open('/Users/olap/Desktop/title_rate.csv', 'r').read().split('\n')
true_label_list = list(map(lambda x:x.split(' ')[-1], ture_label))
true_label_list = [1 if label=='0' else label for label in true_label_list]
true_label_list = [0 if label=='100' else label for label in true_label_list][:-1]

label_list = list(map(lambda x:x.split(',')[-1], title_rate))

In [25]:
title = pd.DataFrame(training_title_list)
feature = pd.DataFrame(title_feature_list)
label = pd.DataFrame(label_list)
true_label = pd.DataFrame(true_label_list)

noobs_df = pd.concat([title, feature, label, true_label], axis= 1)
noobs_df.columns = ['Title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'label', 'true_label']

In [26]:
noobs_df

,Title,token_2,token_3,syn_2,syn_3,label,true_label
0,"大疆发布全新Zenmuse X7云台相机：6K分辨率RAW视频-大疆,Zenmuse X7,...",0,0,1,0,0,0
1,一个出轨女人的后悔自白（在233楼有大家对我提出的疑问进行了详细的解答） - 情感.婚姻.家...,1,0,2,1,1,1
2,跟老公结婚当天，老公强行的让我和他的几个朋友一起爱爱，我是第一次，下面疼的几乎晕过去，屁眼也...,2,1,2,0,1,1
3,"宝马X2量产版曝光：老X1换代重生？-宝马,X2,量产版,X1,SUV,-驱动之家",0,0,1,0,0,0
4,和男友吵架的夜晚心烦意乱，我找了个男人开房解闷,0,1,1,0,1,1
5,日日煮DayDayCook_20170927期-大盘鸡一定要够大盘才尽兴！-生活-高清正版视...,0,0,1,0,0,0
6,充气娃娃真人男用半实体硅胶少妇全自动智能带阴毛成人情趣性用品-tmall.com天猫,4,1,2,1,1,1
7,"蜂巢科技A轮融资6200万,工业级无人机寒冬""起飞""_新闻中心_比特网",0,0,0,0,0,0
8,"美国FBA快递 - 福步外贸网,专注提供各类外贸服务",0,0,0,0,0,0
9,"不省心的民宿短租是该管管了 民宿,短租,旅游,监督 iPhone6|iPhone6 plus...",0,0,0,0,0,0
